In [ ]:
import pandas as pd
df = pd.read_csv("df_neurips_limitation_and_OR_with_cited_data.csv")

openAlex

In [ ]:
import requests

def get_openalex_id_short(title: str) -> str | None:
    """Search OpenAlex by title and return the short ID, or None if not found."""
    resp = requests.get(
        "https://api.openalex.org/works",
        params={"filter": f"title.search:{title}", "per-page": 1}
    )
    resp.raise_for_status()
    results = resp.json().get("results", [])
    if results:
        return results[0]["id"].rsplit("/", 1)[-1]
    return None

def fetch_citers(openalex_id_short: str) -> list[dict]:
    """Given a short OpenAlex ID, return a list of {'title', 'doi'} for each citing work."""
    if not openalex_id_short:
        return []
    resp = requests.get(
        "https://api.openalex.org/works",
        params={
            "filter":   f"referenced_works:{openalex_id_short}",
            "per-page": 200
        }
    )
    resp.raise_for_status()
    citers = resp.json().get("results", [])
    return [
        {"title": c["display_name"], "doi": c.get("doi")}
        for c in citers
    ]

# Prepare the column
df['cited_by'] = None

# Loop and assign in-place
for idx, row in df.iterrows():
    title = row['df_title']
    oa_id = get_openalex_id_short(title)
    citers = fetch_citers(oa_id)
    df.at[idx, 'cited_by'] = citers
    print(f"Row {idx}: found {len(citers)} citers for '{title}'")


In [ ]:
import requests

def get_openalex_id_short(title: str) -> str | None:
    resp = requests.get(
        "https://api.openalex.org/works",
        params={"search": title, "per-page": 1}
    )
    resp.raise_for_status()
    results = resp.json().get("results", [])
    if results:
        return results[0]["id"].rsplit("/", 1)[-1]
    return None

def fetch_citers(openalex_id_short: str) -> list[dict]:
    if not openalex_id_short:
        return []
    resp = requests.get(
        "https://api.openalex.org/works",
        params={"filter": f"referenced_works:{openalex_id_short}", "per-page": 200}
    )
    resp.raise_for_status()
    return [
        {"title": c["display_name"], "doi": c.get("doi")}
        for c in resp.json().get("results", [])
    ]

# ensure the column exists
df['cited_by'] = df.get('cited_by', None)

# only iterate rows from index 102 onward
for idx in df.index[df.index >= 102]:
    title = df.at[idx, 'df_title']
    oa_id  = get_openalex_id_short(title)
    citers = fetch_citers(oa_id)
    df.at[idx, 'cited_by'] = citers
    print(f"Row {idx}: found {len(citers)} citers")



In [ ]:
df.to_csv("df_neurips_limitation_and_OR_with_cited_data.csv",index=False)

arxiv dataset

In [ ]:

import json

data = []
with open(
    "/media/ibrahim/Extreme SSD/Limitations Data/arxiv_metadata/arxiv-metadata-oai-snapshot.json",
    "r", encoding="utf-8"
) as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        # parse each JSON object
        obj = json.loads(line)
        data.append(obj)

# now `data` is a list of dicts
print(len(data), "records loaded")

# turn your list of dicts into a DataFrame
df_arxiv = pd.DataFrame(data)

# a) CSV
df_arxiv.to_csv("arxiv_metadata.csv", index=False)
print("Saved to arxiv_metadata.csv")

2760558 records loaded


In [ ]:
import pandas as pd
df_arxiv = pd.read_csv("arxiv_metadata.csv")

/tmp/ipykernel_7220/920707665.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_arxiv = pd.read_csv("/media/ibrahim/Extreme SSD/Limitations Data/arxiv_metadata/arxiv_metadata.csv")


collect cited papers from arxiv

In [ ]:
# make sure df['cited_by'] already exists and df_arxiv has 'title' and 'id'

for idx, row in df.iterrows():
    citers = row.get('cited_by') or []      # list of dicts
    for citer in citers:
        title_to_find = citer.get('title')
        arxiv_id = None
        # scan every row in df_arxiv for a matching title
        for _, arxiv_row in df_arxiv.iterrows():
            if arxiv_row['title'] == title_to_find:
                arxiv_id = arxiv_row['id']
                print(f"Row {idx}: matched '{title_to_find}' → arXiv ID {arxiv_id}")
                break
        # add the new key (will be None if no match)
        citer['arxiv_id'] = arxiv_id

    # write the updated list of dicts back into df
    df.at[idx, 'cited_by'] = citers

# now each dict in df['cited_by'] has an 'arxiv_id' key
# print(df[['df_title','cited_by']].head())
df.to_csv("df_neurips_limitation_and_OR_with_cited_data.csv",index=False)